In [ ]:
# =========================================
# 0) 설치 (Colab 최초 실행 시)
# =========================================
!pip -q install -U langchain langchain-openai langchain-community langsmith faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 835.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:

# 1) (선택) OpenAI 사용 시
import os, uuid
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "llm_colab_ex_2"                 # 수업용 프로젝트명


In [ ]:


# =========================================
# 2) 문서 예제 준비
# =========================================
docs_text = """
LangSmith는 LLM 애플리케이션을 관찰, 평가, 디버깅할 수 있는 플랫폼입니다.
LangChain은 LLM을 체인으로 구성해 쉽게 개발/운영하도록 돕는 프레임워크입니다.
RAG(Retrieval-Augmented Generation)는 외부 지식을 검색해 답변의 정확성을 높이는 기법입니다.
임베딩(embedding)을 사용해 텍스트를 벡터화하고, 벡터스토어에서 유사도를 기반으로 검색합니다.
"""

# =========================================
# 3) 청킹 (문서 분할)
# =========================================
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=20)
docs = splitter.create_documents([docs_text])

print("✅ 청킹된 문서 개수:", len(docs))
for i, d in enumerate(docs):
    print(f"Chunk {i}:", d.page_content)

# =========================================
# 4) 임베딩 & 벡터스토어 (FAISS)
# =========================================
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)

# =========================================
# 5) Retriever + LLM + Prompt → 체인
# =========================================
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_template(
    "질문: {question}\n"
    "검색된 문서:\n{context}\n"
    "👉 위 문서 내용을 기반으로 한국어로 간결히 답해줘."
)

# 체인 정의 (Runnable 조합)
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# =========================================
# 6) LangSmith 트래킹 실행
# =========================================
config = {
    "tags": ["demo", "rag", "chunking", "embedding"],
    "metadata": {"lesson":"ls-tracing-03"},
    "run_name": "rag_with_chunking_embedding"
}

print(rag_chain.invoke("LangSmith와 RAG는 어떤 관계가 있나요?", config=config))


✅ 청킹된 문서 개수: 4
Chunk 0: LangSmith는 LLM 애플리케이션을 관찰, 평가, 디버깅할 수 있는 플랫폼입니다.
Chunk 1: LangChain은 LLM을 체인으로 구성해 쉽게 개발/운영하도록 돕는 프레임워크입니다.
Chunk 2: RAG(Retrieval-Augmented Generation)는 외부 지식을 검색해 답변의 정확성을 높이는 기법입니다.
Chunk 3: 임베딩(embedding)을 사용해 텍스트를 벡터화하고, 벡터스토어에서 유사도를 기반으로 검색합니다.
LangSmith는 LLM 애플리케이션을 관찰, 평가, 디버깅할 수 있는 플랫폼이며, RAG는 외부 지식을 검색해 답변의 정확성을 높이는 기법입니다. 두 개념은 LLM의 성능을 향상시키기 위해 함께 사용될 수 있습니다.
